In [1]:
import pickle
from operator import itemgetter
import pandas as pd
import plotly.graph_objects as go
import matplotlib
from matplotlib import cm

from hydra.plotly_charts import PlotlyPriceChart, supersample_data
from hydra.utils import split_dupes


# SYMBOLS = ["circle", "square", "diamond-tall", "diamond-wide"]
# COLORS = ["red", "blue", "lime", "goldenrod"]
HEATMAP_COLORS = "afmhot_r"
# filename = "../2019-01-01 - 2020-12-01 1d overlap=0.99 detrend=True.enviro"
filename = "../DOGEUSD 2021-01-01 - 2021-11-19 1d overlap=0.99 detrend=True.enviro"
chart_settings = dict(trendline=False, ifft_price=False, extrapolated=True, showHeatMap=False)

file = None
with open(filename, "rb") as handle:
    file = pickle.load(handle)
interesting, charts, aggregate, inputs = itemgetter(
    "data", "charts", "aggregate", "inputs"
)(file)
window = pd.to_timedelta(inputs["window"])
overlap = inputs["overlap"]
traceCount = min(int(1.0 / (1 - overlap)), len(charts))
kept_frequencies = aggregate["values"]
complexity = aggregate["normalized"]


pair = inputs["pair"]
startDate = charts[0]["startDate"]
endDate = charts[-1]["endDate"]
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"
print(startDate, endDate)
# chart_data = {
#     i: {"trendline": [], "ifft_price": [], "startDate": [], "endDate": []}
#     for i in range(traceCount)
# }

# print(len(charts))
# for index, chart in enumerate(charts):
#     idx = index % traceCount
#     chart_data[idx]["trendline"].append(chart["trendline"])
#     chart_data[idx]["ifft_price"].append(chart["approximated_price"])
    # val, direction = chart['first_extrapolated']
    # chart_data[idx]['extrapolated'].append(val)
    # chart_data[idx]['extrapolated_marker'].append(chart['direction'])
    # chart_data[idx]['startDate'].append(chart['startDate'])
    # chart_data[idx]['endDate'].append(chart['endDate'])

# print(charts[0])
extrapolated_distance_bucket_size = window * 0.01
print(extrapolated_distance_bucket_size)
agg_chart_data = pd.DataFrame(
    {
        "extrapolated": [chart["first_extrapolated"][0] for chart in charts],
        "extrapolated_date": [chart["first_extrapolated"][1] for chart in charts],
        "is_up": [chart["first_extrapolated"][2] for chart in charts],
        "distance": [
            (chart["first_extrapolated"][1] - chart["endDate"]).floor(
                extrapolated_distance_bucket_size
            )
            for chart in charts
        ],
        "startDate": [
            chart['startDate']
            for chart in charts
        ],
        "endDate": [
            chart['endDate']
            for chart in charts
        ],
        # "distance_":[(chart['first_extrapolated'][1] - chart['endDate']) for chart in charts],
        "text": [
            f"{chart['endDate']} ({(chart['first_extrapolated'][1] - chart['endDate'])})<br>isup = {chart['first_extrapolated'][2]}<br>🌊{chart['extrapolated_wavelength']}<br>🔊{chart['extrapolated_amplitude']}"
            for chart in charts
        ],
    }
)
print(agg_chart_data)

agg_chart_data.set_index("extrapolated_date", inplace=True)
agg_data_by_distance = {
    name: {"up": group.loc[group["is_up"]], "down": group.loc[~group["is_up"]]}
    for name, group in agg_chart_data.groupby("distance")
}


# extra_up = agg_chart_data.loc[agg_chart_data["is_up"]]
# extra_down = agg_chart_data.loc[~agg_chart_data["is_up"]]


# print(trendlines)
priceChart = PlotlyPriceChart(pair, startDate, endDate)


def add_trace(
    data,
    chartParams={},
    interval=1,
    useIntervals=[],
    onlySlice=False,
    alwaysUseData=False,
    approximateIntervals=False,
    fields={"x": "index", "y": "values"},
):
    if useIntervals == False:
        priceChart.add_trace(
            go.Scatter(x=data.index, y=data.values, **chartParams),
            onlySlice=onlySlice,
            fields=fields,
        )
        return

    supersampled = supersample_data(
        data,
        interval,
        useIntervals,
        alwaysUseData=alwaysUseData,
        approximate=approximateIntervals,
    )
    priceChart.add_trace(
        go.Scatter(
            x=supersampled[1440].index, y=supersampled[1440].values, **chartParams
        ),
        data=supersampled,
        onlySlice=onlySlice,
        fields=fields,
    )


if chart_settings["ifft_price"]:
    for index in range(traceCount):
        if index in chart_data:
            data = pd.concat(chart_data[index]["ifft_price"])
            add_trace(data, chartParams={"mode": "lines", "name": f"ifft-{index}"})
if chart_settings["trendline"]:
    for index in range(traceCount):
        if index in chart_data:
            data = pd.concat(chart_data[index]["trendline"])
            add_trace(
                data,
                chartParams={
                    "mode": "markers",
                    "name": f"trend-{index}",
                    # "marker_symbol":SYMBOLS[index]
                },
            )

color = cm.get_cmap(HEATMAP_COLORS, len(agg_data_by_distance))
useIntervals = [
    srate
    for srate in [1, 5, 15, 60, 720, 1440]
    if pd.Timedelta(f"{srate * 250 / 30}min") < window
]
print(useIntervals)

if chart_settings["extrapolated"]:
    for num, (key, val) in enumerate(agg_data_by_distance.items()):
        up = val["up"]
        down = val["down"]

        hex = matplotlib.colors.rgb2hex(color(num))
        add_trace(
            up[['extrapolated', 'text', 'endDate']],
            chartParams={
                "mode": "markers",
                "name": f"xtra-u {num * extrapolated_distance_bucket_size}",
                "marker_size": 10,
                "marker_symbol": "triangle-up",
                "marker_color": hex,
                "text": "text",
            },
            onlySlice=True,
            alwaysUseData=True,
            useIntervals=useIntervals,
            fields={"x": "index", "y": "extrapolated", "text":"text"},
        )
        add_trace(
            down[['extrapolated', 'text', 'endDate']],
            chartParams={
                "mode": "markers",
                "name": f"xtra-d {num * extrapolated_distance_bucket_size}",
                "marker_size": 10,
                "marker_symbol": "triangle-down",
                "marker_color": hex,
                "text": "text",
            },
            onlySlice=True,
            alwaysUseData=True,
            useIntervals=useIntervals,
            fields={"x": "index", "y": "extrapolated", "text":"text"},
        )

if chart_settings['showHeatMap']:
    y = complexity.index.tolist()
    # y = [str(float(val).round(4)) for val in y]
    priceChart.add_trace(
        go.Heatmap(
            z=complexity.values.tolist(),
            x=complexity.columns.tolist(),
            y=y,
            showscale=False,
        ),
        loc=(1, 1),
        traceArgs={},
    )
    # priceChart.figure.add_vline(x=startDate, line_width=3)

# Create and add slider
time_slider_steps = []
for i in range(100):
    step = dict(
        method="relayout",
        args=[{"title": f"{i:02d}" + f"% from start (0 = show all)"}],  # layout attribute
    )
    time_slider_steps.append(step)

# frequency_slider_steps = []
# for i in range(len(priceChart.figure.data)):
#     step = dict(
#         method="update",
#         args=[{"visible": [False] * len(priceChart.figure.data)},
#               {"title": "Slider switched to step: " + str(i)}],  # layout attribute
#     )
#     step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
#     time_slider_steps.append(step)

sliders= [dict(
    active=1,
    currentvalue={"prefix": "%Time: "},
    pad={"t": 150},
    steps=time_slider_steps
)]
# ,dict(
#     active=10,
#     currentvalue={"prefix": "Frequency: "},
#     pad={"t": 250},
#     steps=frequency_slider_steps
# )


windowSize = pd.to_timedelta("3h")
currentTime = startDate + windowSize
priceChart.render(sliders=sliders)


2021-01-01 00:00:00 2021-11-18 04:22:00
0 days 00:14:24
       extrapolated   extrapolated_date  is_up        distance  \
0          0.005469 2021-01-02 01:49:00   True 0 days 01:40:48   
1          0.005509 2021-01-02 01:57:00   True 0 days 01:40:48   
2          0.005545 2021-01-02 02:05:00   True 0 days 01:26:24   
3          0.005584 2021-01-02 02:14:00   True 0 days 01:26:24   
4          0.005620 2021-01-02 02:23:00   True 0 days 01:26:24   
...             ...                 ...    ...             ...   
32675      0.237682 2021-11-18 04:07:00  False 0 days 00:28:48   
32676      0.237669 2021-11-18 04:08:00  False 0 days 00:14:24   
32677      0.237626 2021-11-18 04:08:00  False 0 days 00:00:00   
32678      0.237573 2021-11-18 04:09:00  False 0 days 00:00:00   
32679      0.239930 2021-11-18 07:10:00   True 0 days 02:38:24   

                startDate             endDate  \
0     2021-01-01 00:00:00 2021-01-02 00:00:00   
1     2021-01-01 00:14:00 2021-01-02 00:14:00   
2   

FigureWidget({
    'data': [{'close': array([0.0057, 0.0057, 0.0106, 0.0098, 0.0097]),
              'high': a…